# Numba Session 3 exercises

This notebook covers:

* Object mode and loop lifting
  * Inspection using the `numba` tool
* Rewriting NumPy array operations for CUDA kernels using:
  * In-kernel code
  * CuPy
* Asynchronous data movement and kernel launches using streams
* Timing asynchronous operations using events
* CUDA target utility functions:
  * Random Number Generation
  * Reduce Generation
  * ForAll Generation
* Kernel caching and timing
* The CUDA Array Interface

Debugging with the simulator and cuda-memcheck were also part of the Session 3 presentation. There are exercises involving the simulator and cuda-gdb in `2-Debug.ipynb`.

## Prerequisites

Let's first import everything required by the rest of the notebook:

In [ ]:
from IPython.display import HTML
from numba import cuda, jit

import cupy
import hashlib
import numpy as np
import time

## Porting

This section covers useful tools and features when porting code, which is easiest when done in a sequence of stages:

* Compile functions with Object Mode (if necessary)
* Edit Object Mode functions to compile with Nopython Mode
* Move Nopython Mode functions to the GPU with the CUDA JIT decorator
* Optimize data movement
* Optimize computation with asynchronous operations

### Object mode and loop lifting

This is an example of a function that can only be compiled in Object Mode:

In [ ]:
@jit
def hash_computation(x):
    # Loop and body supported by Nopython mode
    for i in range(len(x)):
        x[i] = x[i] ** 2
    # Unsupported library / function call
    return hashlib.md5(x).digest()

When we call it, we can expect to see warnings from Numba:

In [ ]:
x = np.arange(100)
hash_computation(x)

The warnings indicate that the `md5` attribute of the `hashlib` module is unknown to Numba.

Numba lifts the loop that can be compiled in Nopython Mode out of the function and calls it from the Object Mode function, so the code is compiled as if it were written as:

In [ ]:
@jit(nopython=True)
def outline_1(x):
    for i in range(len(x)):
        x[i] = x[i] ** 2


@jit
def hash_computation_internal(x):
    outline_1(x)
    # Unsupported library / function call
    return hashlib.md5(x).digest()

Now let's call the `outline_1` function:

In [ ]:
outline_1(x)

Our success in calling `outline_1` confirms that it could be compiled in Nopython Mode (no warning or error is produced).

Now if we call `hash_computation_internal` we should again see the warning:

In [ ]:
hash_computation_internal(x)

#### Loop Lifting inspection

The `numba` command line tool can be used to produce a report that shows where and why loop lifting occurred. To demonstrate this in the notebook, we first need to write out our above example as a self-contained file:

In [ ]:
%%writefile inspection_example.py
from numba import jit

import hashlib
import numpy as np

@jit
def hash_computation(x):
    # Loop and body supported by Nopython mode
    for i in range(len(x)):
        x[i] = x[i] ** 2
    # Unsupported library / function call
    return hashlib.md5(x).digest()

x = np.arange(100)
hash_computation(x)

Now we can run `numba` over it to generate an HTML report:

In [ ]:
!numba --annotate-html=inspection_example.html inspection_example.py

If we open the report, we should see the lines that can be compiled in Nopython mode highlighted in green, and those that needed Object Mode in red.

Try:
* Clicking the triangle to the left of a source line to expand the Numba IR.
* Look at the line that could not be compiled in Nopython mode - identify the items that are typed as `pyobject`
  * If the code can be modified such that there are no `pyobject` types, it can then be compiled in Nopython mode.
  * For this example, a Nopython-mode compatible implementation of an MD5 function could be substituted for the `hashlib.md5` call (though finding / implementing one may be time consuming - it is not suggested as an exercise for now).

In [ ]:
HTML(filename='inspection_example.html')

### Rewriting NumPy array ops

The following function can be used with Numba's CPU target without issue:

In [ ]:
@jit
def normalize(x):
    for i in range(x.shape[0]):
        x[i, :] = x[i, :] / x[i, :].mean()

Let's create an array on the host and call the function on it:

In [ ]:
x = np.arange(32 * 3).astype(np.float32).reshape((32, 3))
normalize(x)
print(x)

Now we'll try and modify the function to run on the CUDA target. A first approach might consist of:

* Replacing `@jit` with `@cuda.jit`
* Distributing the loop across threads with `cuda.grid(1)`

resulting in something like:

In [ ]:
@cuda.jit
def normalize(x):
    i = cuda.grid(1)
    
    x[i, :] = x[i, :] / x[i, :].mean()

If we try to use this implementation, we have a problem:

In [ ]:
x = np.arange(32 * 3).astype(np.float32).reshape((32, 3))
normalize[1, 32](x)

Numba doesn't recognise the division of an array, by another value, because array operations are not supported in the CUDA target - the kernel needs re-writing to implement the array division, and also the computation of the mean, using only supported mechanisms.

#### Exercises

1. Rewrite the `normalize` kernel so that the array division and mean are computed using for loops and scalar computations with array indexing, so that it can be compiled by Numba.
2. Rewrite the computation in terms of CuPy array operations instead of a Numba kernel.

### Data movement with streams, and timing with events

This next example demonstrates using streams to asynchronously transfer data to and from the device and launch kernels on the streamed data, as well as using events to time these asynchronous events.

First we set things up by creating:

* A kernel to launch
* A stream to operate on
* Host memory allocations using a pinned array - pinned host memory is needed for asynchronous transfers
* Device memory allocations, for the data to be copied to
* Events to mark the start and stop times of asynchronous execution

In [ ]:
@cuda.jit
def increment_kernel(g_data, inc_value):
    i = cuda.grid(1)
    g_data[i] = g_data[i] + inc_value

# Create a stream - differs from CUDA C++ example
stream = cuda.stream()

# Allocate host memory
n = 16 * 1024 * 1024
a = cuda.pinned_array(n, dtype=np.int32)
a[:] = 0

# Allocate device memory
d_a = cuda.device_array_like(a, stream)

# Kernel configuration
nthreads = 512
nblocks = n // nthreads

# Create event handles
start = cuda.event()
stop = cuda.event()

value = 26

After setting everything up, we launch the kernel to ensure that compilation overhead is not part of our timing. We launch the kernel twice, with the second launch undoing the effect of the first:

In [ ]:
# Run the kernel to make sure compilation happens outside the async portion
increment_kernel[nblocks, nthreads, stream](d_a, -value)
increment_kernel[nblocks, nthreads, stream](d_a, value)

Now we can asynchronously issue work. For timing purposes, we first synchronize to ensure that there are no other pending operations when we record the start time. Next, we asynchronously transfer data to the device, launch the kernel, and transfer data back.

The `time()` function is used to record CPU timings; the GPU timings are recorded using the events.

Because all asynchronous operations return immediately, we spin the CPU until all asynchronous operations have completed, by checking for completion with `stop.query()`.

In [ ]:
# Stage 1: Asynchronously issue work
cuda.synchronize()
start_time = time.time()
start.record(stream=stream)
d_a.copy_to_device(a, stream=stream)
increment_kernel[nblocks, nthreads, stream](d_a, value)
d_a.copy_to_host(a)
stop.record(stream=stream)
stop_time = time.time()

# Stage 2: Have CPU do some work while waiting for stage 1 to finish
counter = 0
while not stop.query():
    counter += 1

gpu_time = start.elapsed_time(stop)
cpu_time = (stop_time - start_time) * 1000

Now we can calculate the time spent by the GPU, and the time the CPU spent in CUDA calls. We should be able to see that the CPU was able to do some other work whilst the GPU was executing asynchronously:

In [ ]:
print("time spent executing by the GPU: %.2f" % gpu_time)
print("time spent by CPU in CUDA calls: %.2f" % cpu_time)
print("CPU executed %lu iterations while waiting for GPU to finish" %
      counter)

### Random Number Generation

This section provides a complete example of the use of the CUDA random number generator, by using it to probabilistically compute pi.

First we import functions to initialise the RNG state, and a function to draw random numbers from the distribution:

In [ ]:
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32

Next we'll define the kernel. The pertinent parts of this example are:
* The random number generator states need to be passed into the kernel - here, in `rng_states`.
* The `xoroshiro128p_uniform_float32` function is used to draw a random float32 from a uniform distribution. There are other functions that follow a similar naming convention - for a full list, the [CUDA Random Number Generator documentation](https://numba.pydata.org/numba-doc/latest/cuda/random.html).

In [ ]:
@cuda.jit
def compute_pi(rng_states, iterations, out):
    """Find the maximum value in values and store in result[0]"""
    tid = cuda.grid(1)

    # Compute pi by drawing random (x, y) points and finding what
    # fraction lie inside a unit circle
    inside = 0
    for i in range(iterations):
        x = xoroshiro128p_uniform_float32(rng_states, tid)
        y = xoroshiro128p_uniform_float32(rng_states, tid)
        if x**2 + y**2 <= 1.0:
            inside += 1

    out[tid] = 4.0 * inside / iterations

Prior to using the RNG, we need to initialize its states. The state size should be equal to the total number of threads in the grid.

In [ ]:
nthreads = 64
nblocks = 24    
state_size = nblocks * nthreads
rng_states = create_xoroshiro128p_states(state_size, seed=1)

Now we can call the kernel using our initialized states:

In [ ]:
out = np.zeros(nthreads * nblocks, dtype=np.float32)
compute_pi[nblocks, nthreads](rng_states, 10000, out)
print('pi:', out.mean())

### The Reduce Generator

The reduction generator simplifies the generation of fast reduction kernels. It accepts a function defined in terms of a pair of elements to reduce, and generates a kernel that applies the reduction to all elements of an input array. We can create a reduction using the `@cuda.reduce decorator` - here we will create a simple sum-reduction, but more complex computations are possible:

In [ ]:
@cuda.reduce
def sum_reduce(a, b):
    return a + b

Now let's create an array to test the reduction:

In [ ]:
A = (np.arange(1234, dtype=np.float64)) + 1

If we call the reduction function on an array residing on the host, it automatically transfers data to the device, and transfers the result back to return it:

In [ ]:
sum_reduce(A)

If we want to keep the result on the device, we need a device array to store the result:

In [ ]:
r = cuda.device_array(1, dtype=np.float64)
sum_reduce(A, res=r)

Calling a reduction kernel on a device array does not overwrite the device array (the documentation presently states, in error, that it does). We'll try with a device array:

In [ ]:
# Does not(!) overwrite device array
A_d = cuda.to_device(A)
sum_reduce(A_d, res=r)

Now if we check if `A_d` remains the same as it was when it was copied from the host:

In [ ]:
np.all(np.allclose(A, A_d.copy_to_host()))

### The ForAll Generator

The ForAll generator takes a kernel that operates elementwise on an input array and returns a preconfigured kernel ready to launch on an array of a given size. We'll try it out with an example borrowed from the internals of [cuDF](https://github.com/rapidsai/cudf), which rounds elements to a given number of decimal places.

First, we'll define our kernels:

In [ ]:
from math import floor

@cuda.jit(device=True)
def rint(x):
    """Round to the nearest integer.

    Returns
    -------
    The nearest integer, as a float.
    """
    y = floor(x)
    r = x - y 

    if r > 0.5:
        y += 1.0 
    if r == 0.5:
        r = y - 2.0 * floor(0.5 * y)
        if r == 1.0:
            y += 1.0 
    return y


@cuda.jit
def gpu_round(in_col, out_col, decimal):
    i = cuda.grid(1)
    f = 10 ** decimal

    if i < in_col.size:
        ret = in_col[i] * f
        ret = rint(ret)
        tmp = ret / f
        out_col[i] = tmp

Now we'll prepare some data - an input and output array, and note the number of elements:

In [ ]:
# in_data and out_data are device arrays
in_data = cuda.to_device(np.random.random(10))
out_data = cuda.device_array_like(in_data)
nelems = len(in_data)

To apply the kernel to all elements of the array without having to work out an efficient launch configuration we call the `forall` method of the kernel, then call the result with the usual kernel arguments. The `forall` method calculates occupancy and generates an efficient launch configuration automatically:

In [ ]:
# Round all elements to 3 d.p.
gpu_round.forall(nelems)(in_data, out_data, 3)

We can print the input data and results to check that the kernel did what we expected:

In [ ]:
print(in_data.copy_to_host())
print(out_data.copy_to_host())

## Performance

### Caching

During the first execution of a jitted function, Numba steps in and compiles the Python code, whereas subsequent executions retrieve a compiled version from a cache. This compilation time can be quite long in comparison to a single function/kernel call. When measuring the execution time of jitted functions, either on the CPU or GPU, it is important to avoid timing the compilation as well as the execution time.

Timing the compilation can be avoided by only timing the second or later call to a jitted function. We'll see the difference in times with a small example kernel. The following defines the kernel, and sets up some data to call it with:

In [ ]:
@jit(nopython=True)
def go_fast(a):
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace


x = np.arange(4096).astype(np.float64).reshape((64, 64))

If we time the first execution, we will get a measurement of the compilation time and the first execution time:

In [ ]:
# Compilation time included in the first execution
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %sms" % ((end - start) * 1000))

A subsequent execution should require much less time:

In [ ]:
# Compilation time included in the first execution
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %sms" % ((end - start) * 1000))

For this particular example, the difference in timing is likely to be orders of magnitude.

## The CUDA Array Interface

This section demonstrates:

* How the CUDA Array Interface enables interoperability between different CUDA-supporting Python libraries from the user's perspective with two examples:
  - Calling a Numba kernel on a CuPy array
  - Using CuPy on a Numba device array
* How to implement the CUDA Array Interface in a Python library, using a simple example that wraps the Runtime API.


### Calling a Numba kernel on a CuPy array

We'll define a simple CUDA kernel that adds two arrays:

In [ ]:
@cuda.jit
def add(x, y, out):
    start = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

Now we'll create some CuPy arrays for input data and space for an output:

In [ ]:
a = cupy.arange(10)
b = a * 2
out = cupy.zeros_like(a)

Without needing to do anything else from the user perspective, we can call our Numba kernel on CuPy arrays:

In [ ]:
add[1, 32](a, b, out)

and get the correct result:

In [ ]:
print(a+b)
print(out)

This worked because Numba used the CUDA Array Interface to access data at the time of the launch - seeing objects other than its own device arrays, it accessed the `__cuda_array_interface__` property of the arrays to obtain a pointer to the data and the shape and strides of the arrays.

### Creating CuPy arrays from Numba device arrays without copying

Now let's work in the other direction - we'll create a CuPy array view of a Numba device array, so that we can operate on it with CuPy functions. First let's create a Numba array:

In [ ]:
# type: numpy.ndarray
x = np.arange(10)

# type: numba.cuda.cudadrv.devicearray.DeviceNDArray
x_numba = cuda.to_device(x)

Now we can create a CuPy array and compute the mean:

In [ ]:
# type: cupy.ndarray - a view of x_numba's data
x_cupy = cupy.asarray(x_numba)

x_cupy.mean()

If we mutate the Numba array on the device then computing the mean again will use the updated values, because `x_numba` and `x_cupy` share the underlying data:

In [ ]:
@cuda.jit
def add_one(x):
    i = cuda.grid(1)
    if i < len(x):
        x[i] += 1

# Add one to every element of the array
add_one[1, 32](x_numba)

x_cupy.mean()

### Building a CUDA Array Interface implementation

Implementing the CUDA Array Interface so a library can export its data for other CUDA-aware libraries to use requires the implementation of one property, `__cuda_array_interface__`, on the objects that hold on-device data. This property returns a dict containing information about the data. For full details of the specification, see the [CUDA Array Interface documentation](http://numba.pydata.org/numba-doc/latest/cuda/cuda_array_interface.html).

We'll implement the interface for a simple class that allocates and frees on-device memory using the CUDA Runtime API via ctypes.

First we'll set up our ctypes access to the Runtime API:

In [ ]:
# Get access to the CUDA Runtime API via ctypes

from ctypes import CDLL, POINTER, byref, c_void_p, c_size_t

cudart = CDLL('libcudart.so')

cudaMalloc = cudart.cudaMalloc
cudaMalloc.argtypes = [POINTER(c_void_p), c_size_t]

cudaFree = cudart.cudaFree
cudaFree.argtypes = [c_void_p]

Now we'll define the `MyFloatArray` class:

In [ ]:
# Create a class to provide the CUDA Array interface.
#
# Simple example for read-write contiguous data.

FLOAT32_SIZE = 4

class MyFloatArray:
    """An array of floats that allocates memory on the device when constructed,
    and frees it when it is deleted.
    """

    def __init__(self, size):
        self._size = size
        self._typestr = 'f4'
        self._ptr = c_void_p()

        alloc_size = size * FLOAT32_SIZE
        ret = cudaMalloc(byref(self._ptr), alloc_size)
        if ret:
            raise RuntimeError(f'Unexpected return code {ret} from cudaMalloc')

    def __del__(self):
        cudaFree(self._ptr)

    @property
    def __cuda_array_interface__(self):
        return {
            # Fill in
        }

#### Exercise

* Fill in code to return the entries in the dict returned by `__cuda_array_interface__`

#### Continuing

Once you have completed filling in the dict entries we can create an instance of the class:

In [ ]:
nelems = 32
arr = MyFloatArray(nelems)

For checking things, we can access the CUDA Array Interface manually and print various fields:

In [ ]:
arr_cai = arr.__cuda_array_interface__

cai_ptr = arr_cai['data'][0]
print(f"Pointer from CAI is 0x{cai_ptr:x}")

print(f"Shape from CAI is {arr_cai['shape']}")
print(f"Type string from CAI is '{arr_cai['typestr']}'")
print(f"Version from CAI is {arr_cai['version']}")

If everything looks as expected, we can launch a kernel on our `MyFloatArray` object transparently:

In [ ]:
@cuda.jit
def initialize(x):
    i = cuda.grid(1)
    if i < len(x):
        x[i] = 3.14


initialize[1, 32](arr)

We can also copy the data to the host and print it out to check that it is expected. For simplicity in this example, we'll use the `as_cuda_array()` function, that takes any object and creates a Numba device array from it, so that we can copy the data and print it.

In [ ]:
print(cuda.as_cuda_array(arr).copy_to_host())

We should see that the array is populated with the value `3.14` in every element.

A more thorough / complete test might have used the Runtime API to copy the data to the host then print it, but this would require some extra work in this example.

## Debugging

Session 3 also covered debugging techniques. For exercises with cuda-memcheck and the CUDA simulator, see the other notebook, `2-Debug.ipynb`.

## Summary

In this notebook, we've looked at how to:

* Determine what is compiling in Object Mode and why by using the `numba` command line tool to create HTML source code listings annotated with Numba type information,
* Rewrite array operations unsupported in the CUDA target, both with loops and scalar code in kernels, and using CuPy functions,
* How to launch asynchronous transfers and kernels, and how to measure their performance using events,
* How to use the CUDA target's utility functions:
  - The Random Number Generator,
  - The Reduce Generator,
  - The ForAll Generator,
* How to time jitted functions and kernel launches without including the compilation time,
* How to use the CUDA array interface:
  - From the usage perspective, with CuPy and Numba data,
  - and how to implement the CUDA Array Interface for an object holding on-device data.